In [1]:
take = 1

# Description

Script to run the agent on the hardware

0. Description
1. Imports
2. Paths
3. Configuration
4. Functions
5. Creation
6. Run on Hardware
7. Save recorded Data


# Specification

In [2]:
control_rate = 75 #Hz
dt = 0.1
model_number = 999
max_limit = 0.0001 #0.0001 0.000085
R1 = 1
R2 = 0.005
R_success = 250
reality_distance_threshold = 0.0075

reality_horizon = 3000

# Imports

In [3]:
# General imports
import os
import time
import datetime
from os.path import expanduser
import pandas
import pickle
import json
import numpy
from matplotlib import pyplot as plt

# rllib imports
import ray
import ray.rllib.agents.ppo as ppo
from ray.tune.registry import get_trainable_cls
from ray.tune.registry import register_env
from ray.rllib.env import MultiAgentEnv
from ray.rllib.env.base_env import _DUMMY_AGENT_ID
from ray.rllib.evaluation.worker_set import WorkerSet
from ray.rllib.policy.sample_batch import DEFAULT_POLICY_ID
from ray.rllib.utils.deprecation import deprecation_warning

# Gym imports
from gym.envs.robotics.ur10 import randomize
from gym.envs.robotics.ur10 import pos_randomize
from gym.envs.robotics import ur10_noisy_pd_env_mh
from gym.envs.robotics import ur10_sim2real_pos_env
from gym import utils
from gym.envs.robotics import rotations, robot_custom_env_mh, utils
from gym.envs.robotics import ur10_force_env_3
from gym.envs.robotics import ur10_force_env

# MuJoCo imports
import mujoco_py

# ROS imports
import rospy
from sensor_msgs.msg import JointState
from std_msgs.msg import Float64MultiArray
from std_msgs.msg import String
from geometry_msgs.msg import WrenchStamped
from trajectory_msgs.msg import JointTrajectory, JointTrajectoryPoint
from realsense2_camera.msg import CustomVector
from realsense2_camera.msg import EstimatedPose

In /home/lars/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/lars/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/lars/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/lars/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/lars/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_c

# Paths

In [4]:
HOME_PATH = os.getenv("HOME")
PROJECT_PATH = os.path.join(*[HOME_PATH, "DRL_AI4RoMoCo"])
SAVE_PATH = os.path.join(*[PROJECT_PATH, "code", "data","TEST_Reality"])

#XML_MODEL_PATH = os.path.join(*[HOME_PATH, "DRL_AI4RoMoCo", "code", "environment","UR10_single","ur10_heg.xml"])   #Force
XML_MODEL_PATH = os.path.join(*[HOME_PATH, "DRL_AI4RoMoCo", "code", "environment","UR10_single_position","ur10_heg_position.xml"])   #Position

env_config = "reality_config_force.yml"
#checkpoint_number = 4950
#model = "PPO_NoisyPositionEnv_0_2020-09-18_20-14-55ej_cw9mo"
control_type = env_config.split("_")[2].split(".")[0]

ENV_CONFIG = os.path.join(*[HOME_PATH, "AGENT_TESTS", env_config])
#ENV_CONFIG = os.path.join(*[HOME_PATH, "AGENT_TESTS", "PPO_ForceEnv_0_2020-09-10_15-29-51_j1bhegb", config])

# Configuration

Hardware - Environment (ROS)

In [5]:
actual_q = numpy.zeros(6,)
ft_values = numpy.zeros(6,)

Simulation - Environment (MuJoCo, Gym)

In [6]:
goal = numpy.array([0.69423743, -0.83110109,  1.17388998, -1.57161506,  0.02185773, -3.14102438])
reward_type = "dense"
distance_threshold = 0.002
fail_threshold = 0.25
punish_force = False
punish_force_thresh = 20
punish_force_factor = 0.001
print(goal)

[ 0.69423743 -0.83110109  1.17388998 -1.57161506  0.02185773 -3.14102438]


In [7]:
initial_qpos_simpheg_conf2 = numpy.array([0, -1.3, 2.1, -0.80, 1.5708, 0])
randomize_kwargs = {
    'var_mass':     0,#0.5,
    'var_damp':     0,#0.5,
    'var_fr':       0,#0.5,
    'var_grav_x_y': 0,#0.5,
    'var_grav_z':   0,#1,
    'var_body_pos': 0,#0.05,
    'var_body_rot': 0,
}

Agent (ray, tune, rllib)

In [8]:
checkpoint_model1 = os.path.join(*[HOME_PATH, "DRL_SetBot-RearVentilation", "final_models",
                            "model1", "checkpoint-2120"])
checkpoint_model2 = os.path.join(*[HOME_PATH, "DRL_SetBot-RearVentilation", "final_models",
                            "model2", "checkpoint-3440"])
checkpoint_model3 = os.path.join(*[HOME_PATH, "DRL_SetBot-RearVentilation", "final_models",
                            "model3", "checkpoint-3030"])
checkpoint_model4 = os.path.join(*[HOME_PATH, "DRL_SetBot-RearVentilation", "final_models",
                            "model4", "checkpoint-3590"])
checkpoint_model5 = os.path.join(*[HOME_PATH, "DRL_SetBot-RearVentilation", "final_models",
                            "model5", "checkpoint-4410"])

#checkpoint_lars = os.path.join(*[HOME_PATH, "AGENT_TESTS", "PPO_NoisyVaryEnv_0_2020-09-15_13-53-11d046bzlf", "checkpoint_2675", "checkpoint-2675"]) #Fail
#checkpoint_lars = os.path.join(*[HOME_PATH, "AGENT_TESTS", "PPO_NoisyVaryEnv_0_2020-09-16_20-33-05bjvfub2b", "checkpoint_4275", "checkpoint-4275"]) #works
#checkpoint_lars = os.path.join(*[HOME_PATH, "AGENT_TESTS", "PPO_NoisyPositionEnv_0_2020-09-18_20-14-55ej_cw9mo", "checkpoint_4950", "checkpoint-4950"]) #Position
#checkpoint_lars = os.path.join(*[HOME_PATH, "AGENT_TESTS", "PPO_NoisyEnv_0_2020-09-22_19-50-24bt7wx44r", "checkpoint_1175", "checkpoint-1175"]) # New and corrective
#checkpoint_lars = os.path.join(*[HOME_PATH, "AGENT_TESTS", "PPO_NoisyEnv_0_2020-09-22_13-53-53m9s7_v8d", "checkpoint_220", "checkpoint-220"]) # New and corrective
#checkpoint_lars = os.path.join(*[HOME_PATH, "AGENT_TESTS", "PPO_NoisyEnv_0_2020-09-17_16-12-44a70jmqsn", "checkpoint_3150", "checkpoint-3150"]) #works
#checkpoint_lars = os.path.join(*[HOME_PATH, "AGENT_TESTS", "PPO_NoisyCorEnv_0_2020-09-22_12-24-02irctporb", "checkpoint_75", "checkpoint-75"]) # CorEnv

#scheckpoint_lars = os.path.join(*[HOME_PATH, "AGENT_TESTS", "PPO_StaticEnv_0_2020-09-24_08-39-32jdnl7ykv", "checkpoint_810", "checkpoint-810"]) # Static25
checkpoint_lars = os.path.join(*[HOME_PATH, "AGENT_TESTS", "PPO_NoisyEnv_0_2020-09-24_09-07-22vaoju9fy", "checkpoint_780", "checkpoint-780"]) # Noisy25
#checkpoint_lars = checkpoint_model4    # Gutes Modell


In [9]:
def update_checkpoint_for_rollout(checkpoint_path):
    with open(checkpoint_path, "rb") as f:
        extra_data = pickle.load(f)
    if not "trainer_state" in extra_data:
        extra_data["trainer_state"] = {}
        with open(checkpoint_path, 'wb') as f:
            pickle.dump(extra_data, f)

checkpoint = checkpoint_lars
update_checkpoint_for_rollout(checkpoint)
checkpoint

'/home/lars/AGENT_TESTS/PPO_NoisyEnv_0_2020-09-24_09-07-22vaoju9fy/checkpoint_780/checkpoint-780'

In [10]:
config = {}
    # Load configuration from checkpoint file.
config_dir = os.path.dirname(checkpoint)
config_path = os.path.join(config_dir, "params.pkl")
# Try parent directory.
if not os.path.exists(config_path):
    config_path = os.path.join(config_dir, "../params.pkl")
if os.path.exists(config_path):
    print("Config found!")
    with open(config_path, "rb") as f:
            config = pickle.load(f)
            
config['num_workers'] = 2
config['num_gpus'] = 1

Config found!


In [11]:
config

{'env': 'NoisyEnv',
 'seed': 1794,
 'num_workers': 2,
 'num_envs_per_worker': 1,
 'num_gpus': 1,
 'gamma': 0.99,
 'lr': 0.0003,
 'horizon': 500,
 'soft_horizon': False,
 'train_batch_size': 11000,
 'callbacks': {'on_episode_start': None,
  'on_episode_step': None,
  'on_episode_end': None,
  'on_postprocess_traj': None},
 'model': {'use_lstm': False,
  'lstm_cell_size': 128,
  'fcnet_hiddens': [256, 256]},
 'tf_session_args': {'allow_soft_placement': True}}

# Functions

Helper Functions

In [12]:
def normalize_rad(angles):
    angles = numpy.array(angles)
    angles = angles % (2*numpy.pi)
    angles = (angles + 2*numpy.pi) % (2*numpy.pi)
    for i in range(len(angles)):
        if (angles[i] > numpy.pi):
            angles[i] -= 2*numpy.pi
    return angles

def get_obs():
    set_state(actual_q)
    for i in range(10):
        viewer.render()
    #print(ft_values)
    x_pos = sim.data.get_body_xpos("gripper_dummy_heg")
    x_mat = sim.data.get_body_xmat("gripper_dummy_heg")
    rpy =  normalize_rad(rotations.mat2euler(x_mat))
    #print(x_pos, rpy)

    obs = numpy.concatenate([
                x_mat.dot(x_pos-goal[:3]), x_mat.dot(normalize_rad(rpy-goal[3:])), ft_values.copy()
            ])
    return obs

def get_dq(dx):
    jacp = sim.data.get_body_jacp(name="gripper_dummy_heg").reshape(3, 6)
    jacr = sim.data.get_body_jacr(name="gripper_dummy_heg").reshape(3, 6)
    jac = numpy.vstack((jacp, jacr))
    dq = numpy.linalg.lstsq(jac, dx)[0].reshape(6, )
    return dq

def go_smooth(q):
    rate = rospy.Rate(125)
    dq = q - actual_q
    while numpy.linalg.norm(dq)>0.005:
        dq = q - actual_q
        dq_ = numpy.clip(dq, -0.0001, 0.0001)
        control(ctrl_q+dq_)
        rate.sleep()
    control(q)
    
def set_state(qpos):
    #assert qpos.shape == (model.nq,)
    old_state = sim.get_state()
    new_state = mujoco_py.MjSimState(old_state.time, qpos, old_state.qvel,
                                     old_state.act, old_state.udd_state)
    sim.set_state(new_state)
    sim.forward()
    
def keep_going(steps, num_steps, episodes, num_episodes):
    """Determine whether we've collected enough data"""
    # if num_episodes is set, this overrides num_steps
    if num_episodes:
        return episodes < num_episodes
    # if num_steps is set, continue until we reach the limit
    if num_steps:
        return steps < num_steps
    # otherwise keep going forever
    return True

ROS

In [13]:
# ROS callback functions
def q_callback(data):
    '''Callback for the joint angles, velocities and accelerations'''
    global actual_q
    global actual_qd
    global actual_qdd
    actual_q = numpy.array(data.position)[[2,1,0,3,4,5]]
    actual_qd = numpy.array(data.velocity)[[2,1,0,3,4,5]]
    actual_qdd = numpy.array(data.effort)[[2,1,0,3,4,5]]
    set_state(actual_q)
    viewer.render()
    
def ft_callback(data):
    '''Callback for the Robotiq FT300 force torque values'''
    global ft_values
    ft_values = 1 * numpy.array([data.wrench.force.x, data.wrench.force.y, data.wrench.force.z, data.wrench.torque.x, \
                            data.wrench.torque.y, data.wrench.torque.z])
    
def pose_callback(data):
    '''Callback for the Intel Realsense D435i and OpenCV charuco pose estimation'''
    global pose
    pose = numpy.array([data.tx, data.ty, data.tz, data.rx, data.ry, data.rz])
    
def observation_callback(data):
    '''Callback for the entire observation array (same as in simulation)'''
    global observations
    observations = numpy.array(data.data)
    
def check_ROS_topics():
    '''Listens to all required ROS topics and checks if each of them is online and sending'''
    check_jointstates = []
    check_ft300_force_torque = []
    check_observations = []
    check_pose_estimation = []
    
    for i in range(5):
        check_jointstates.append(actual_q)
        check_ft300_force_torque.append(ft_values)
        check_observations.append(observations)
        check_pose_estimation.append(pose)
        time.sleep(0.5)
        
    topics = ["/joint_states", "/ft300_force_torque", "/observation", "/pose_estimation"]
    topics_dict = {
        "/joint_states" : check_jointstates,
        "/ft300_force_torque" : check_ft300_force_torque,
        "/observation" : check_observations,
        "/pose_estimation" : check_pose_estimation
    }
    
    for topic in topics:
        if not topics_dict[topic]:
            print("ERROR: {} is offline or not sending".format(topic))
        else:
            print("SUCCESS: {} is online and sending".format(topic))

In [14]:
def goal_distance(obs, goal):
    '''Compute distance between TCP and Goal'''
    obs = obs[:6]
    assert obs.shape == goal.shape
    return numpy.linalg.norm(obs*numpy.array([1, 1, 1, 0.3, 0.3, 0.3]), axis=-1)

def compute_reward(obs, goal):
    d = goal_distance(obs,goal)
    f = numpy.absolute(obs[7]) + numpy.absolute(obs[8]) + numpy.absolute(obs[9])
    rew = R1 * (-d) + R2 *(-f)
    return rew
'''
def is_success(obs, goal):
    d = goal_distance(obs,goal)
    if d < distance_threshold:
        return True
    else:
        return False
'''
def is_success(obs):
    distance = numpy.absolute(obs[0]) + numpy.absolute(obs[1]) + numpy.absolute(obs[2])
    if distance < reality_distance_threshold:
        return True
    else:
        return False

In [15]:
# ROS control commands
def control(q):
    '''Controlling the Universal Robot UR10 JointGroupPositionController'''
    global ctrl_q
    command = Float64MultiArray()
    command.data = q
    control_publisher.publish(command)
    ctrl_q = q.copy()
    
def traj_control(q):
    #dt = 0.025
    global ctrl_q
    command = JointTrajectory()
    point = JointTrajectoryPoint()
    
    #command.header.stamp = rospy.Time.now()
    #command.header.frame_id = "DRL_UR10"
    #command.joint_names = ["elbow_joint", "shoulder_lift_joint", "shoulder_pan_joint", "wrist_1_joint", "wrist_2_joint", "wrist_3_joint"]
    
    #command.header.stamp = rospy.Time.now()
    command.header.frame_id = "DRL_UR10"
    command.joint_names.append("shoulder_pan_joint")
    command.joint_names.append("shoulder_lift_joint")
    command.joint_names.append("elbow_joint")
    command.joint_names.append("wrist_1_joint")
    command.joint_names.append("wrist_2_joint")
    command.joint_names.append("wrist_3_joint")
    

    point.positions.append(q[0])
    point.positions.append(q[1])
    point.positions.append(q[2])
    point.positions.append(q[3])
    point.positions.append(q[4])
    point.positions.append(q[5])
    
    command.points.append(point)
    command.points[0].time_from_start=rospy.Duration.from_sec(dt)
    
    traj_control_publisher.publish(command)
    ctrl_q = q.copy()
    
def static_approach():
    
    waypoint0 = numpy.array([1.6256170272827148, -0.9155753294574183, 1.5307668447494507, -0.6977546850787562, 1.5475375652313232, 0.011224725283682346])[[2,1,0,3,4,5]]
    waypoint1 = numpy.array([2.47381329536438, -1.4324410597430628, 1.5743204355239868, -1.0292657057391565, 1.5910834074020386, 0.011859889142215252])[[2,1,0,3,4,5]]
    waypoint2 = numpy.array([2.519468069076538, -1.6841238180743616, 0.13313426077365875, -0.8226664702044886, 1.7688652276992798, 0.014099815860390663])[[2,1,0,3,4,5]]
    waypoint3 = numpy.array([2.1001405715942383, -1.3000262419330042, 7.190534961409867e-05, -0.7999804655658167, 1.5709056854248047, -3.5587941304981996e-05])[[2,1,0,3,4,5]]
    
    dt_static = 2
    command = JointTrajectory()
    point0 = JointTrajectoryPoint()
    point1 = JointTrajectoryPoint()
    point2 = JointTrajectoryPoint()
    point3 = JointTrajectoryPoint()
    
    command.header.stamp = rospy.Time.now()
    command.header.frame_id = "DRL_UR10"
    command.joint_names.append("shoulder_pan_joint")
    command.joint_names.append("shoulder_lift_joint")
    command.joint_names.append("elbow_joint")
    command.joint_names.append("wrist_1_joint")
    command.joint_names.append("wrist_2_joint")
    command.joint_names.append("wrist_3_joint")
    
    point0.positions.append(waypoint0[0])
    point0.positions.append(waypoint0[1])
    point0.positions.append(waypoint0[2])
    point0.positions.append(waypoint0[3])
    point0.positions.append(waypoint0[4])
    point0.positions.append(waypoint0[5])
    
    #command.points.append(point0)
    
    point1.positions.append(waypoint1[0])
    point1.positions.append(waypoint1[1])
    point1.positions.append(waypoint1[2])
    point1.positions.append(waypoint1[3])
    point1.positions.append(waypoint1[4])
    point1.positions.append(waypoint1[5])
    
    command.points.append(point1)
    command.points[0].time_from_start=rospy.Duration.from_sec(dt_static)
    
    point2.positions.append(waypoint2[0])
    point2.positions.append(waypoint2[1])
    point2.positions.append(waypoint2[2])
    point2.positions.append(waypoint2[3])
    point2.positions.append(waypoint2[4])
    point2.positions.append(waypoint2[5])
    
    command.points.append(point2)
    command.points[1].time_from_start=rospy.Duration.from_sec(2*dt_static)
    
    point3.positions.append(waypoint3[0])
    point3.positions.append(waypoint3[1])
    point3.positions.append(waypoint3[2])
    point3.positions.append(waypoint3[3])
    point3.positions.append(waypoint3[4])
    point3.positions.append(waypoint3[5])
    
    command.points.append(point3)
    command.points[2].time_from_start=rospy.Duration.from_sec(2.5*dt_static)
    
    traj_control_publisher.publish(command)
    
def ft_zero():
    '''Zeroes the Robotiq FT300 force torque sensor by sending an ur_script via UR Secondary Interface'''
    zero_sensor_urscript = 'sec ft_set_zero():' + '\n' + '\t' + 'if(socket_open("127.0.0.1", 63350, "acc")):' \
          + '\n' + '\t' + '\t' + 'socket_send_string("SET ZRO", "acc")' \
         + '\n' + '\t' + '\t' + 'socket_close("acc")' + '\n' + '\t' + 'end' + '\n' + 'end' + '\n'
    ft_zero_publisher.publish(zero_sensor_urscript)

# Creation

Simulation Environment (Gym)

In [16]:
model = mujoco_py.load_model_from_path(XML_MODEL_PATH)
sim = mujoco_py.MjSim(model)
q_init = numpy.array([0, -1.3, 2.1, -0.80, 1.5708, 0.0])

ctrl_q = q_init
set_state(q_init)

viewer = mujoco_py.MjViewer(sim)
viewer.render()

Creating window glfw


In [17]:
x_pos = sim.data.get_body_xpos("gripper_dummy_heg")
x_mat = sim.data.get_body_xmat("gripper_dummy_heg")
rpy =  normalize_rad(rotations.mat2euler(x_mat))
print(x_pos, rpy)

[ 0.57243453 -0.83605904  1.17575472] [-1.57079633e+00  3.67320510e-06  3.14159265e+00]


In [18]:
if control_type == "force":
    class Ur10HegRandForceEnv(ur10_noisy_pd_env_mh.Ur10Env):
        def __init__(self, env_config):
            model_xml_path = os.path.join(*[PROJECT_PATH, "code", "environment", "UR10_Force_Randomized", "ur10_assembly_setup_rand_temp_{}.xml".format(env_config.worker_index)])
            randomize.randomize_ur10_xml(worker_id = env_config.worker_index)
            ur10_noisy_pd_env_mh.Ur10Env.__init__(
                self, ENV_CONFIG, model_xml_path = model_xml_path, worker_id=env_config.worker_index
            )
            #utils.EzPickle.__init__(self)
    register_env("NoisyEnv", lambda config: Ur10HegRandForceEnv(config))
    print("FORCE CONTROL")
elif control_type == "position":
    class Ur10HegRandPositionEnv(ur10_sim2real_pos_env.Ur10Env):
        def __init__(self, env_config):
            model_xml_path = os.path.join(*[PROJECT_PATH, "code", "environment", "UR10_Position_Randomized", "ur10_assembly_setup_rand_temp_{}.xml".format(env_config.worker_index)])
            pos_randomize.randomize_ur10_xml(worker_id = env_config.worker_index)
            ur10_sim2real_pos_env.Ur10Env.__init__(
                self, ENV_CONFIG, model_xml_path = model_xml_path, worker_id=env_config.worker_index
            )
            #utils.EzPickle.__init__(self)
    register_env("NoisyPositionEnv", lambda config: Ur10HegRandPositionEnv(config))
    print("POSITION CONTROL")
else:
    print("ERROR for control_type:", control_type)

FORCE CONTROL


Agent

In [19]:
ray.init()
cls = get_trainable_cls("PPO")

2020-09-25 10:36:34,818	INFO resource_spec.py:212 -- Starting Ray with 15.48 GiB memory available for workers and up to 7.76 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-09-25 10:36:35,445	INFO services.py:1083 -- View the Ray dashboard at localhost:8265


In [20]:
#agent = cls(env="RandomizedEnv", config=config)
agent = cls(config=config)
agent.restore(checkpoint)

2020-09-25 10:36:36,865	INFO trainer.py:420 -- Tip: set 'eager': true or the --eager flag to enable TensorFlow eager execution
2020-09-25 10:36:36,947	INFO trainer.py:580 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
/home/lars/gym/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


(pid=25978) /home/lars/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=25978)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=25978) /home/lars/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=25978)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=25978) /home/lars/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=25978)   _np_qint16 = np.dtype([("qint16", np.int16, 1)])
(pid=25978) /home/lars/.l

2020-09-25 10:36:43,536	WARNING trainable.py:210 -- Getting current IP.
2020-09-25 10:36:43,537	INFO trainable.py:416 -- Restored on 10.180.134.127 from checkpoint: /home/lars/AGENT_TESTS/PPO_NoisyEnv_0_2020-09-24_09-07-22vaoju9fy/checkpoint_780/checkpoint-780
2020-09-25 10:36:43,538	INFO trainable.py:423 -- Current state after restoring: {'_iteration': 780, '_timesteps_total': None, '_time_total': 83469.00706005096, '_episodes_total': 55618}


(pid=25975) In /home/lars/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
(pid=25975) The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
(pid=25975) In /home/lars/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
(pid=25975) The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
(pid=25975) In /home/lars/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
(pid=25975) In /home/lars/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
(pid=25975) The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases lat

In [21]:
print(ray)
print(get_trainable_cls)
print(ray.tune.registry)
clss = get_trainable_cls("PPO")

<module 'ray' from '/home/lars/.local/lib/python3.6/site-packages/ray/__init__.py'>
<function get_trainable_cls at 0x7f5b6de9c048>
<module 'ray.tune.registry' from '/home/lars/.local/lib/python3.6/site-packages/ray/tune/registry.py'>


ROS-nodes

In [22]:
rospy.init_node("DRL_AI4RoMoCo", anonymous=True)
rospy.Subscriber("/joint_states", JointState, q_callback)
rospy.Subscriber("/ft300_force_torque", WrenchStamped, ft_callback)
rospy.Subscriber("/pose_estimation", EstimatedPose, pose_callback)
rospy.Subscriber("/observation", CustomVector, observation_callback)
#control_publisher = rospy.Publisher("/joint_group_position_controller/command", Float64MultiArray, queue_size=1)
traj_control_publisher = rospy.Publisher("/pos_traj_controller/command", JointTrajectory, queue_size=1)
ft_zero_publisher = rospy.Publisher("/ur_hardware_interface/script_command", String, queue_size=1)

#rate = rospy.Rate(1)
rate = rospy.Rate(control_rate)

(pid=25975) /home/lars/gym/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=25975)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=25978) /home/lars/gym/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=25978)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Data structures for storing and saving data

In [23]:
observations

array([ 0.12803569,  0.00244428,  0.00488203, -0.00085671, -0.00045745,
       -0.02183218,  0.04      ,  0.08      ,  0.02      ,  0.004     ,
        0.003     ,  0.001     ])

In [24]:
observations_rec = []
poses_rec = []
ft_values_rec = []
q_rec = []
qd_rec = []
rewards_rec = []
step = 0

# Run On Hardware

In [25]:
agent.compute_action([0,0,0,0,0,0,0,0,0,0,0,0])

array([-1.        , -0.65675116,  1.        , -0.16475676, -0.59990036,
        0.10499126], dtype=float32)

In [ ]:
done = False
ft_zero()
#max_limit = 0.00075

time.sleep(2.5)
while not done:
    obs = get_obs()
    action = agent.compute_action(obs)
    for i in range(1):
        dx = max_limit * action.copy()
        rot_mat = sim.data.get_body_xmat('gripper_dummy_heg')
        dx_ = numpy.concatenate([rot_mat.dot(dx[:3]), rot_mat.dot(dx[3:])])  ## transform to right coordinate system
        dq  = get_dq(dx_)
        q = ctrl_q + dq
        traj_control(q)
        print(observations[6:9])
        observations_rec.append(observations)
        poses_rec.append(pose)
        ft_values_rec.append(ft_values)
        q_rec.append(actual_q)
        qd_rec.append(actual_qd)
        rewards_rec.append(compute_reward(obs, goal))
        step += 1
    if is_success(obs) or step >= reality_horizon:
        rewards_rec.append(R_success)
        break
    rate.sleep()

/home/lars/.local/lib/python3.6/site-packages/ipykernel_launcher.py:29: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


[0.09       0.09       0.18000001]
[0.09       0.09       0.18000001]
[0.06       0.13       0.27000001]
[0.06       0.13       0.27000001]
[-0.03  0.04  0.16]
[-0.03  0.04  0.16]
[-0.03  0.04  0.16]
[-0.05  0.07  0.08]
[-0.05  0.07  0.08]
[0.13 0.03 0.07]
[0.13 0.03 0.07]
[0.13 0.03 0.07]


/home/lars/.local/lib/python3.6/site-packages/ipykernel_launcher.py:29: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


[-0.22        0.25        0.23999999]
[-0.22        0.25        0.23999999]
[-0.22        0.25        0.23999999]
[0.58999997 0.18000001 0.05      ]
[0.58999997 0.18000001 0.05      ]
[0.15000001 0.16       0.05      ]
[0.15000001 0.16       0.05      ]
[0.15000001 0.16       0.05      ]
[0.04       0.46000001 0.28      ]
[0.04       0.46000001 0.28      ]
[ 0.34999999  0.23       -0.18000001]
[ 0.34999999  0.23       -0.18000001]
[-0.13        0.47        0.25999999]
[-0.13        0.47        0.25999999]
[-0.13        0.47        0.25999999]
[0.69999999 0.31999999 0.02      ]
[0.69999999 0.31999999 0.02      ]
[0.12       0.50999999 0.12      ]
[0.12       0.50999999 0.12      ]
[ 0.44  0.2  -0.01]
[ 0.44  0.2  -0.01]
[ 0.44  0.2  -0.01]
[0.18000001 0.33000001 0.02      ]
[0.18000001 0.33000001 0.02      ]
[0.18000001 0.33000001 0.02      ]
[0.09 0.16 0.07]
[ 0.15000001 -0.06        0.08      ]
[ 0.15000001 -0.06        0.08      ]
[ 0.15000001 -0.06        0.08      ]
[-0.02        0

/home/lars/.local/lib/python3.6/site-packages/ipykernel_launcher.py:29: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


...

[0.95999998 0.69999999 0.62      ]
[0.95999998 0.69999999 0.62      ]
[0.95999998 0.69999999 0.62      ]
[0.63999999 0.89999998 1.07000005]
[0.63999999 0.89999998 1.07000005]
[0.68000001 1.25       0.73000002]
[0.68000001 1.25       0.73000002]
[0.68000001 1.25       0.73000002]
[0.62       1.10000002 0.99000001]
[0.62       1.10000002 0.99000001]
[0.66000003 0.57999998 0.61000001]
[0.66000003 0.57999998 0.61000001]
[0.66000003 0.57999998 0.61000001]
[0.91000003 1.19000006 0.99000001]
[0.91000003 1.19000006 0.99000001]
[0.58999997 0.97000003 0.51999998]
[0.58999997 0.97000003 0.51999998]
[0.58999997 0.97000003 0.51999998]
[1.00999999 1.05999994 1.01999998]
[1.00999999 1.05999994 1.01999998]
[0.70999998 0.74000001 0.81999999]
[0.70999998 0.74000001 0.81999999]
[0.70999998 0.74000001 0.81999999]
[0.57999998 0.74000001 0.75      ]
[0.57999998 0.74000001 0.75      ]
[1.05999994 1.24000001 0.82999998]
[1.05999994 1.24000001 0.82999998]
[1.05999994 1.24000001 0.82999998]
[1.27999997 0.94    

/home/lars/.local/lib/python3.6/site-packages/ipykernel_launcher.py:29: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


[0.85000002 0.89999998 0.81999999]
[0.85000002 0.89999998 0.81999999]
[0.85000002 0.89999998 0.81999999]
[0.85000002 0.89999998 0.81999999]
[0.85000002 0.89999998 0.81999999]
[0.85000002 0.89999998 0.81999999]
[0.85000002 1.00999999 0.77999997]
[0.85000002 1.00999999 0.77999997]
[0.66000003 0.81999999 0.81999999]
[0.66000003 0.81999999 0.81999999]
[0.66000003 0.81999999 0.81999999]
[0.69999999 0.89999998 0.85000002]
[0.69999999 0.89999998 0.85000002]
[0.89999998 1.07000005 0.82999998]
[0.89999998 1.07000005 0.82999998]
[0.63       1.13999999 0.89999998]
[0.63       1.13999999 0.89999998]
[0.63       1.13999999 0.89999998]
[0.58999997 0.93000001 1.        ]
[0.58999997 0.93000001 1.        ]
[0.58999997 1.01999998 0.82999998]
[0.58999997 1.01999998 0.82999998]
[0.58999997 1.01999998 0.82999998]
[0.81999999 0.93000001 0.63      ]
[0.81999999 0.93000001 0.63      ]
[0.81999999 0.93000001 0.63      ]
[0.79000002 0.88999999 0.92000002]
[0.79000002 0.88999999 0.92000002]
[0.79000002 0.839999

/home/lars/.local/lib/python3.6/site-packages/ipykernel_launcher.py:29: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.



[0.61000001 0.88999999 0.77999997]
[0.75999999 0.95999998 0.89999998]
[0.75999999 0.95999998 0.89999998]
[0.75999999 0.95999998 0.89999998]
[0.75999999 0.91000003 0.58999997]
[0.75999999 0.91000003 0.58999997]
[0.61000001 0.89999998 0.77999997]
[0.61000001 0.89999998 0.77999997]
[0.61000001 0.89999998 0.77999997]
[0.68000001 0.88       0.82999998]
[0.68000001 0.88       0.82999998]
[0.56999999 0.88999999 0.77999997]
[0.56999999 0.88999999 0.77999997]
[0.56999999 0.88999999 0.77999997]
[0.70999998 0.89999998 0.76999998]
[0.74000001 0.97000003 0.86000001]
[0.74000001 0.97000003 0.86000001]
[0.81       0.94999999 0.77999997]
[0.81       0.94999999 0.77999997]
[0.75       0.88       0.76999998]
[0.75       0.88       0.76999998]
[0.75       0.88       0.76999998]
[0.75       0.88       0.76999998]
[0.75       0.88       0.76999998]
[0.86000001 0.75999999 0.72000003]
[0.86000001 0.75999999 0.72000003]
[0.76999998 0.93000001 0.80000001]
[0.76999998 0.93000001 0.80000001]
[0.76999998 0.93000

# !!! Save recorded Data !!!

In [26]:
txs = []
tys = []
tzs = []
rxs = []
rys = []
rzs = []
fxs = []
fys = []
fzs = []
txs = []
tys = []
tzs = []

for i in range(len(observations_rec)):

    txs.append(observations_rec[i][0])
    tys.append(observations_rec[i][1])
    tzs.append(observations_rec[i][2])
    rxs.append(observations_rec[i][3])
    rys.append(observations_rec[i][4])
    rzs.append(observations_rec[i][5])
    fxs.append(observations_rec[i][6])
    fys.append(observations_rec[i][7])
    fzs.append(observations_rec[i][8])
    txs.append(observations_rec[i][9])
    tys.append(observations_rec[i][10])
    tzs.append(observations_rec[i][11])
    
recordings = {
    "tx" : txs,
    "ty" : tys,
    "tz" : tzs,
    "rx" : rxs,
    "ry" : rys,
    "rz" : rzs,
    "fx" : fxs,
    "fy" : fys,
    "fz" : fzs,
    "tx" : txs,
    "ty" : tys,
    "tz" : tzs,
    "Reward" : rewards_rec
}

In [27]:
filename = "{}".format(take).zfill(4)+"_model_{}.json".format(model_number)
with open(os.path.join(*[SAVE_PATH, filename]), 'w') as file:
    json.dump(recordings, file)
    file.close()

In [28]:
observations_rec_df = pandas.DataFrame(observations_rec, columns=["pos1", "pos2", "pos3", "rotx", "roty", "rotz", "Fx", "Fy", "Fz", "Mx", "My", "Mz"])
poses_rec_df = pandas.DataFrame(poses_rec, columns=["tx", "ty", "tz", "rx", "ry", "rz"])
ft_values_rec_df = pandas.DataFrame(ft_values_rec, columns=["Fx", "Fy", "Fz", "Mx", "My", "Mz"])
q_rec_df = pandas.DataFrame(q_rec, columns=["q[0]", "q[1]", "q[2]", "q[3]", "q[4]", "q[5]"])
qd_rec_df = pandas.DataFrame(qd_rec, columns=["qd[0]", "qd[1]", "qd[2]", "qd[3]", "qd[4]", "qd[5]"])
rewards_df = pandas.DataFrame(rewards_rec, columns=["Rewards"])
info_df = pandas.DataFrame(columns=['Steps', 'Reward'], index=[model_name+"_"+str(take)])

info_df.loc[model_name+"_"+str(take), "Steps"] = len(rewards_rec)
info_df.loc[model_name+"_"+str(take), "Reward"] = numpy.sum(rewards_rec)
    
DIRECTORY = os.path.join(*[SAVE_PATH])
observations_rec_df.to_feather(os.path.join(*[SAVE_PATH, "{}".format(take).zfill(4)+"_obs.ftr"]))
poses_rec_df.to_feather(os.path.join(*[SAVE_PATH, "{}".format(take).zfill(4)+"_pose.ftr"]))
ft_values_rec_df.to_feather(os.path.join(*[SAVE_PATH, "{}".format(take).zfill(4)+"_ft.ftr"]))
q_rec_df.to_feather(os.path.join(*[SAVE_PATH, "{}".format(take).zfill(4)+"_q.ftr"]))
qd_rec_df.to_feather(os.path.join(*[SAVE_PATH, "{}".format(take).zfill(4)+"_qd.ftr"]))
rewards_df.to_feather(os.path.join(*[SAVE_PATH, "{}".format(take).zfill(4)+"_reward.ftr"]))
info_df = info_df.reset_index()
info_df.to_feather(os.path.join(*[SAVE_PATH, "{}".format(take).zfill(4)+"_info.ftr"]))

NameError: name 'model_name' is not defined

# --- Visualize the Data with Visualize_Hardware_Tests.ipynb ---